**Содержание**<a id='toc0_'></a>    
- [Многопоточное программирование](#toc1_)    
  - [Асинхронное выполнение](#toc1_1_)    
  - [std::async](#toc1_2_)    
    - [Пример: MapReduce](#toc1_2_1_)    
  - [std::thread](#toc1_3_)    
    - [Пример: MapReduce](#toc1_3_1_)    
  - [Синхронизация](#toc1_4_)    
  - [std::atomic](#toc1_5_)    
  - [Общие советы и замечания](#toc1_6_)    
    - [Примеры](#toc1_6_1_)    
- [Коллекция библиотек Boost](#toc2_)    
  - [Категории библиотек Boost](#toc2_1_)    
- [Метапрограммирование: основы](#toc3_)    
  - [Метапрограммирование](#toc3_1_)    
  - [Метафункции](#toc3_2_)    
  - [Вычисления в compile-time](#toc3_3_)    
  - [Определение списка](#toc3_4_)    
  - [Длина списка](#toc3_5_)    
  - [Примеры применения шаблонов с переменным числом аргументов](#toc3_6_)    
  - [Операции со списком](#toc3_7_)    
    - [Задачи](#toc3_7_1_)    
  - [Вывод списка](#toc3_8_)    
    - [Задачи](#toc3_8_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Многопоточное программирование](#toc0_)

До С++11 многопоточности не было в стандарте, т.к. формально он был однопоточным. Все реализации многопоточности были на основе расширений компилятора и возможностей ОС, т.е. за пределами стандарта.

## <a id='toc1_1_'></a>[Асинхронное выполнение](#toc0_)

Предположим, что мы хотим вычислить `doAsyncWork` асинхронно.

    int doAsyncWork();

В C++ есть два способа выполнения задач асинхронно:

∙ создать поток вручную `std::thread`,

    #include <thread>
    // создание потока, вычисляющего doAsyncWork()
    std::thread t(doAsyncWork);

∙ использование `std::async`.

    #include <future>
    // использование std::async
    std::future<int> fut = std::async(doAsyncWork);
    int res = fut.get();


Замечания:

`std::async` возвращает не поток, а объект `std::future` - который в дальнейшем позволит нам получить доступ к возвр.значению функции `doAsyncWork` используя его методы, например `get` (дождаться возврата из `doAsyncWork` и вернуть значение).

`std::async` может в некоторых случаях (зависит от планировщика) отложить выполнение задачи до вызова `get` или `wait`, т.е. выполнить ее фактически синхронно, без создания отдельного потока

`std::thread` не имеет стандартных способов возвращения значений из потока, это нужно писать самому

## <a id='toc1_2_'></a>[std::async](#toc0_)

∙ Имеет две стратегии выполнения: **асинхронное** выполнение и **отложенное** (синхронное) выполнение.

1. `std::launch::async`
2. `std::launch::deferred` (когда вызывается `future.get`)

∙ По умолчанию имеет стратегию: `std::launch::async | std::launch::deferred` (т.е. на усмотрение планировщика, который сам решает, если уже много потоков, то может отложить создание нового)

    // гарантирует асинхронное выполнение
    std::future<int> fut =
        std::async(std::launch::async, doAsyncWork);
    int res = fut.get();

∙ Отложенная задача может никогда не выполниться, если не будет вызвано `get` или `wait`.

∙ Возвращает `std::future<T>`, который позволяет получить возвращаемое значение.

∙ Позволяет обрабатывать исключения. Если в `doAsyncWork` произошло исключение, то оно сохраняется, и при вызове `get` выбрасывается в наш поток.

∙ Деструктор `future` блокирует поток до тех пор, пока не получит возвращаемое значение, и с виду асинхронный код на деле может оказаться синхронным

### <a id='toc1_2_1_'></a>[Пример: MapReduce](#toc0_)

В данной задаче от вас потребуется реализовать аналог популярной концепции MapReduce в терминах потоков. MapReduce - модель распределённых вычислений, представленная компанией Google, используемая для параллельных вычислений над очень большими наборами данных в компьютерных кластерах.

Напишите функцию `map_reduce`, которая принимает на вход:

- итератор на начало последовательности,
- итератор на конец последовательности,
- унарный функтор, который можно применить к элементам последовательности,
- бинарный функтор, который можно применить к результатам применения первого функтора,
- количество потоков.

Результат вызова `map_reduce`

    auto res = map_reduce(p, q, f1, f2, num);

должен быть эквивалентен результату следующего кода:

    auto res = f1(*p);
    while(++p != q)
        res = f2(res, f1(*p));

только выполнение этого кода должно быть разбито на `num` потоков. Для этого можно разбить диапазон `[p,q)` на `num` частей, выполнить вычисление для каждой части в отдельном потоке и объединить результаты из всех потоков.

Пример:

    std::list<int> l = {1,2,3,4,5,6,7,8,9,10};
    // параллельное суммирование в 3 потока
    auto sum = map_reduce(l.begin(), l.end(), 
                 [](int i){return i;}, 
                        std::plus<int>(), 3);

    // проверка наличия чётных чисел в четыре потока
    auto has_even = map_reduce(l.begin(), l.end(), 
                     [](int i){return i % 2 == 0;}, 
                    std::logical_or<bool>(), 4);
    
Гарантии:

- Длина последовательности не меньше количества потоков.
- Функция f2 ассоциативна (т.е. можно тоже запихнуть в потоки и делать частичные свертки).
- последовательность не меняется и не копируется
- у потоков нет разделяемых данных, поэтому синхронизация не требуется

Примерно так

    template <typename It, typename F1, typename F2>
    auto sync_map_reduce(It p, It q, F1 f1, F2 f2) 
    -> decltype(f2(f1(*p), f1(*p)))
    {
        auto res = f1(*p);
        while (++p != q)
            res = f2(res, f1(*p));
        return res;
    }

    template <typename It, typename F1, typename F2>
    auto map_reduce(It p, It q, F1 f1, F2 f2, size_t threads)
    -> decltype(f2(f1(*p),f1(*p)))                                          // так удобнее всего вывести тип результат
    { 
        using TRes = decltype(f2(f1(*p),f1(*p)));                           // создадим для него алиас

        size_t length = std::distance(p, q);
        
        std::vector<std::future<TRes>> vecFut;                              // промисы

        for (size_t i = 0; i < threads; i++) {
            auto p1 = p;
            auto q1 = p;
            std::advance(p1, int(i * length / threads));                    // сдвинутые итераторы
            std::advance(q1, int((i + 1) * length / threads));
            
            // промисы нельзя копировать, поэтому только так
            vecFut.push_back(std::async(std::launch::async, sync_map_reduce<It, F1, F2>, p1, q1, f1, f2));
        }
    
        // в C++14 можно auto вместо std::future<TRes>&
        auto res = sync_map_reduce(vecFut.begin(), vecFut.end(), [](std::future<TRes>& fut) {return fut.get(); }, f2); 
        return res;
    }

Затыки:
- опция компилятора `-lpthread` !
- промисы нельзя копировать, поэтому сразу в `push_back`

## <a id='toc1_3_'></a>[std::thread](#toc0_)

∙ Сразу же начинает вычислять переданную функцию (ОС создается системный поток).

∙ Игнорирует возвращаемое значение функции. Нужен разделяемый объект для получения результата, например, захватить по ссылке локальную переменную:

    // переменная для возвращаемого значения
    int res = 0;
    std::thread t([&res](){res = doAsyncWork();});
    t.join();

∙ Метод `join()` позволяет заблокировать текущий поток, пока выполнение потока не завершится.

∙ Метод `detach()` позволяет отключить поток от объекта, т.е. разорвать связь между объектом и потоком.
- порожденный поток будет работать в фоне и если основной поток завершится до его окончания, то это считается UB (поток станет зомби, а UB потому что мало ли чего он там в фоне может наворотить без основного потока)

∙ При вызове деструктора подключаемого потока (т.е. если поток завершился до `join/detach`) программа завершается (`std::terminate`), т.е. необходимо вызывать `join` или `detach`

∙ Исключения не могут покидать пределы потока.
- если исключение покидает порожденный поток, то основной поток получает `std::terminate`

∙ `native_handle()` возвращает дескриптор потока по объекту потока.
- можно реализовать какие-то возможности, которые стандарт не предоставляет, например, использовать для дескриптора какие-то вызовы из системных библиотек

например `man 2 kill`

    #include <sys/types.h>
    #include <signal.h>
    
    ...
    kill(t.native_handle(), SIGKILL);

### <a id='toc1_3_1_'></a>[Пример: MapReduce](#toc0_)

Напишите функцию map_reduce из предыдущей задачи, но с использованием `std::thread`.

    template <typename It, typename F1, typename F2>
    auto sync_map_reduce(It p, It q, F1 f1, F2 f2) 
    -> decltype(f2(f1(*p), f1(*p)))
    {
        auto res = f1(*p);
        while (++p != q)
            res = f2(res, f1(*p));
        return res;
    }

    template <typename It, typename F1, typename F2>
    auto map_reduce(It p, It q, F1 f1, F2 f2, size_t threads)
    -> decltype(f2(f1(*p),f1(*p)))                                          // так удобнее всего вывести тип результат
    { 
        using TRes = decltype(f2(f1(*p),f1(*p)));                           // создадим для него алиас
        using TF1res  = decltype(f1(*p));

        size_t length = std::distance(p, q);
        
        std::vector<std::thread> ts;                                        // потоки
        std::vector<TRes> results(threads);                                 // результаты

        for (size_t i = 0; i < threads; i++) {
            auto p1 = p;
            auto q1 = p;
            std::advance(p1, int(i * length / threads));                    // сдвинутые итераторы
            std::advance(q1, int((i + 1) * length / threads));
            
            std::thread t( [&results, i, p1, q1, f1, f2](){ results[i] = sync_map_reduce(p1, q1, f1, f2); } );
            
            ts.push_back(std::move(t));
        }

        for (auto & t : ts)
            t.join();
    
        auto res = sync_map_reduce(results.begin(), results.end(), [](TF1res x){return x;}, f2);

        return res;
    }

Затыки:
- по ссылке в поток нужно захватывать только изменяемые потоком переменные. Захват индекса по ссылке `&i` в лямбду приводит к тому, что когда поток выполнит функцию и захочет записать результат `i` уже может уехать в другое место
- объект потока нельзя копировать, ну хоть мувить можно -> добавление в вектор перемещением 

## <a id='toc1_4_'></a>[Синхронизация](#toc0_)

Нужна только тогда, когда разные потоки обращаются к общим данным

    double shared = 0; // разделяемая переменная
    std::mutex mtx; // мьютекс для shared

    void compute(int begin, int end) {
        for (int i = begin; i != end; ++i) {
            double current = someFunction(i);
    
            // критическая секция
            std::lock_guard<std::mutex> lck(mtx);
            shared += current;
        }
    }
    
    int main () {
        std::thread th1 (compute, 0, 100);
        std::thread th2 (compute, 100, 200);
        th1.join();
        th2.join();
     
        std::cout << shared << std::endl;
    }

`std::lock_guard` - захватывает мьютекс. Это RAII обертка над мьютексом обеспечивающая строгую гарантию безопасности исключение. Можно использовать интерфейс самого мьютекса, методы `lock/unlock`, но они такой гарантии не дают.
- ее критическая секция в данном случае - это одна строка, т.к. `std::lock_guard` действует до вызова своего деструктора и здесь он - локальная переменная цикла, т.к. живет только 1 итерацию
- явно вызывать деструктор `std::lock_guard` для разблокировки **неправильно**, всегда можно обернуть обращение к разделяемому ресурсу в отдельный блок (или функцию). Отдельный блок может быть использован и просто для того, чтобы выделить какую-то смысловую часть кода.

после джойнов потоки зевершились и к разделяемой переменной можно обращаться без локов

## <a id='toc1_5_'></a>[std::atomic](#toc0_)

Используется когда разделяемый ресурс это одна встроенная целочисленная переменная или указатель.

∙ Шаблон `std::atomic` позволяет определить переменную, операции с которой будут атомарны.

∙ Определён только для целочисленных встроенных типов и указателей.
- если точнее, то можно использовать и для других POD-типов (Plain Old Data - структура или класс без конструкторов и т.п.), но тогда атомарность уже может обеспечиваться не на уровне процессора (если размер типа больше регистра процессора)

Пример, разделяемый указатель - атомарный счетчик ссылок

    template<class T>
    struct shared_ptr_data
    {
        void addref()
        {
            ++counter; // atomic increment
        }
        
        T * ptr;
        std::atomic<size_t> counter;
    };

Главный плюс в том, что в отличие от мьютексов, где блокировку обеспечиват ОС своими системными вызовами, тут атомарность гарантируется на уровне процессора, что обычно работает **существенно** быстрее

## <a id='toc1_6_'></a>[Общие советы и замечания](#toc0_)

∙ Предпочитайте `std::async` прямому созданию потоков.

∙ Гарантируйте неподключённость потоков на всех путях выполнения (в т.ч. при возникновении исключений).
- почему-то неподключённостью тут назвали вызов `join/detatch`

∙ При использовании `std::thread` следите за тем, чтобы исключения не покидали функцию потока.

∙ Используйте `std::atomic` вместо мьютекса, когда синхронизация нужна только для одной целочисленной переменной.

∙ Делайте константные методы безопасными в смысле потоков (например, при кешировании)
- обычно специально для этого делать ничего не надо, но если метод сложный, включащий неконстантные операции, то с ними также надо заботиться о синхронизации

∙ `volatile` - это не про многопоточность, объявление такой переменной никак не сказываеться на обеспечении синхронизации ее разделения между потоками

### <a id='toc1_6_1_'></a>[Примеры](#toc0_)

Нужна ли синхронизация мьютексами?

- Несколько потоков читают одну переменную типа string. (НЕТ)
- Несколько потоков изменяют одну переменную типа string. (ДА)
- Несколько потоков работают с одной переменной типа int: один изменяет, а остальные читают. (НЕТ, сделать атомарной)
- Несколько потоков работают с одной переменной типа string: один изменяет, а остальные читают. (ДА)
- Несколько потоков работают с одним vector<string>, но при этом количество элементов в нём не изменяется. (ДА)
- Несколько потоков работают с одним vector<string>, количество элементов не изменяется, каждый поток работает со своим элементом. (НЕТ)

# <a id='toc2_'></a>[Коллекция библиотек Boost](#toc0_)


∙ Это коллекция библиотек, расширяющих функциональность C++.

∙ Свободно распространяются по лицензии Boost Software License вместе с исходным кодом и документацией на www.boost.org.

∙ Лицензия позволяет использовать boost в коммерческих проектах.

∙ Библиотеки из boost являются кандидатами на включение в стандарт C++.

∙ Некоторые библиотеки boost были включены в стандарты C++ 2011/14 года (std::function, std::thread, . . .).

∙ При включении библиотеки в boost она проходит несколько этапов рецензирования.

∙ Библиотеки boost позволяют обеспечить переносимость.

∙ В текущей версии в boost более сотни библиотек.


## <a id='toc2_1_'></a>[Категории библиотек Boost](#toc0_)

- String and text processing
- Containers,
- Iterators
- Algorithms
- Function objects and higher-order programming
- Generic Programming
- Template Metaprogramming
- Concurrent Programming
- Math and numerics
- Correctness and testing
- Data structures
- Domain Specific
- System
- Input/Output
- Memory
- Image processing
- Inter-language support
- Language Features Emulation
- Parsing
- Patterns and Idioms
- Programming Interfaces
- State Machines
- Broken compiler workarounds
- Preprocessor Metaprogramming


Примеры

`boost::any` позволяет хранить значения любого типа, который можно копировать и присваивать

`boost::optional` позволяет хранить значение одного типа, либо  специальный тип - "ничего".

`boost::variant` позволяет хранить значение одного из наперёд заданных типов.

`boost::any` для хранения значения выделяет динамическую память.

Что позволяет сделать `boost::filesystem`?
- Обрабатывать пути и имена файлов и директорий.
- Работать с файлами и их аттрибутами.
- Работать с директориями и их аттрибутами.

в C++17 вошли в станарт

Какая библиотека из коллекции boost позволяет легко написать http-сервер
- Asio


# <a id='toc3_'></a>[Метапрограммирование: основы](#toc0_)

## <a id='toc3_1_'></a>[Метапрограммирование](#toc0_)

∙ Метапрограммированием называют создание программ, которые порождают другие программы.

∙ Шаблоны C++ можно рассматривать как функциональный язык для метапрограммирования (порождает "безшаблонные" программы на языке C++).

∙ Метапрограммы C++ позволяют оперировать типами, шаблонами и значениями целочисленных типов.

∙ Метапрограммирование в C++ можно применять для широкого круга задач:

- целочисленные compile-time вычисления,
- compile-time проверка ошибок,
- условная компиляция,
- генеративное (обобщенное) программирование - парадигма программирования, такое описание данных и алгоритмов, которое можно применять к различным типам данных, не меняя само это описание,
- . . .

∙ Для метапрограммирования существуют целые библиотеки, например, MPL из boost.

Нужно понимать, что есть функциональные языки программирования (Haskell и т.д.), где все сделано по задуманному, а в С++ - это больше похоже на "так получилось" и в принципе работает

## <a id='toc3_2_'></a>[Метафункции](#toc0_)

Аналог функции в метапрограммировании, функция высшего порядка, принимает "функцию", возвращает "функцию"

Метафункция — в С++ это шаблонный класс, но не любой, а такой, который определяет либо имя типа `type`, либо целочисленную константу `value`.

- Аргументы метафункции — это аргументы шаблона.
- Возвращаемое значение — это type или value.

Метафункции могут возвращать типы:

    template<typename T>
    struct AddPointer
    {
        using type = T *;
    };

    int main() {
        AddPointer<int>::type p = new int(0);
        delete p;
        return 0;
    }

и значения целочисленных типов:

    template<int N>
    struct Square
    {
        static int const value = N * N;
    };

*) static чтобы value была известна на этапе компиляции. Аналогичный эффект может быть получен

    template<typename T, T N>
    constexpr T Square = N * N;

## <a id='toc3_3_'></a>[Вычисления в compile-time](#toc0_)

Пример, что можно делать в compile-time

    template<int N>
    struct Fact
    {
        static int const value = N * Fact<N - 1>::value;
    };

    template<>
    struct Fact<0>
    {
        static int const value = 1;
    };

    int main()
    {
        std::cout << Fact<10>::value << std::endl;
    }

Ценность этого, что в скомпилированном коде в данном месте уже будет подставлено конкретное значение, которое не надо вычислять, а сами шаблоны ни во что не компилируются, потому что шаблоны


(такие простые compile-time вычисления проще реализовать через constexpr)

Еще пример

    // Напишите метафункцию Fib<N>, которая вычисляет N-ое число Фибоначчи
    template<int N>
    struct Fib
    { static int const value = Fib<N - 1>::value + Fib<N - 2>::value; };

    template<>
    struct Fib<0>
    { static int const value = 0; };

    template<>
    struct Fib<1>
    { static int const value = 1; };


## <a id='toc3_4_'></a>[Определение списка](#toc0_)

Шаблоны позволяют определять **алгебраические типы данных** (список - такой тип).

    // определяем список (объявление шаблонной структуры с переменным числом шаблонных аргументов)
    template <typename ... Types>
    struct TypeList;

    // специализация по умолчанию (хотя бы один тип, он будет Head)
    template <typename H, typename... T>
    struct TypeList<H, T...>
    {
        using Head = H;
        using Tail = TypeList<T...>;
    };
    
    // специализация для пустого списка (полная специализация шаблона)
    template <>
    struct TypeList<> { };

Пример: **список целых чисел времени компиляции**

    // Определение списка целых чисел времени компиляции IntList
    template <int ... Types>
    struct IntList;

    template <int H, int... T>
    struct IntList<H, T...>
    {
        static int const Head = H;
        using Tail = IntList<T...>;
    };

## <a id='toc3_5_'></a>[Длина списка](#toc0_)

    // вычисление длины списка
    template<typename TL>
    struct Length
    {
        static int const value = 1 +
        Length<typename TL::Tail>::value;
    };

    template<>
    struct Length<TypeList<>>
    {
        static int const value = 0;
    };

    int main()
    {
        using TL = TypeList<double, float, int, char>;
        std::cout << Length<TL>::value << std::endl;
        return 0;
    }

Пример: **список целых чисел времени компиляции** (меняется 1 токен)

    // вычисление длины списка
    template<typename TL>
    struct Length
    {
        static int const value = 1 +
        Length<typename TL::Tail>::value;
    };

    template<>
    struct Length<IntList<>>
    { static int const value = 0; };

## <a id='toc3_6_'></a>[Примеры применения шаблонов с переменным числом аргументов](#toc0_)

Стоит упомянуть, что шаблоны с переменным числом аргументов позволяют делать следующие странные вещи.

Наследовать класс от списка параметров шаблона

    // класс Derived будет унаследован от своих шаблонных параметров

    template <typename... BaseClasses> 

    struct Derived : BaseClasses... 
    {
    // конструктор класса Derived будет 
    // принимать значения типа своих базовых классов
    Derived (BaseClasses const&... base_classes) 
        // и создавать их при помощи конструктора копирования
        : BaseClasses(base_classes)... 
    {}
    };

Узнавать количество аргументов.

    template<typename ...Args> 

    struct SomeStruct 
    {

    // это не размер Args, а количество аргументов в Args
    static const int count = sizeof...(Args); 
    };

Поэтому длину списка можно было бы реализовать проще непосредственно в классе TypeList.

    // определяем список
    template <typename ... Types>
    struct TypeList; 

    // специализация по умолчанию
    template <typename H, typename... T>
    struct TypeList<H, T...> 
    {
        using Head = H;
        using Tail = TypeList<T...>;
        static const int Length = 1 + sizeof...(T);
    };

    // специализация для пустого списка
    template <>
    struct TypeList<> 
    { 
        static const int Length = 0;
    };

## <a id='toc3_7_'></a>[Операции со списком](#toc0_)

Словом `cons` в функц.ЯП часто обозначают добавление элемента в начало списка

    // добавление элемента в начало списка
    template<typename H, typename TL>
    struct Cons;

    template<typename H, typename... Types>
    struct Cons<H, TypeList<Types...>>          // уот так уот мы получим сами типы, которые в TypeList
    {
        using type = TypeList<H, Types...>;     // тогда результат это просто новый TypeList
    };

    // конкатенация списков (уаще аналогично)
    template<typename TL1, typename TL2>
    struct Concat;
    
    template<typename... Ts1, typename... Ts2>
    struct Concat<TypeList<Ts1...>, TypeList<Ts2...>>
    {
        using type = TypeList<Ts1..., Ts2...>;
    };

Чтобы было все lisp-ориентировано, можно было определить cons как создание пары. А list, как создание вложенных cons-ов.

### <a id='toc3_7_1_'></a>[Задачи](#toc0_)

Напишите две метафункции для работы c IntList:

- IntCons позволяет увеличить список на один элемент — он добавляется в начало списка.
- Generate позволяет сгенерировать список длины N с числами от 0 до N - 1.

Вариант

    // реализация метафункции IntCons позволяет увеличить список на 
    // один элемент — он добавляется в начало списка
    template<int H, typename TL>
    struct IntCons;

    template<int H, int... values>
    struct IntCons<H, IntList<values...>>
    {
        using type = IntList<H, values...>;
    };

    // генерация списка
    template <int N, int ... Next>
    struct Generate : public Generate<N-1, N-1, Next...>
    { };

    template <int ... Next>
    struct Generate<0, Next ... >
     { using type = IntList<Next ... >; };
    
    // вариант генерации
    template<size_t N, int K = 0>
    struct Generate
    { using type = typename IntCons<K, typename Generate<N - 1, K + 1>::type>::type; };

    template<int K>
    struct Generate<0, K>
    { using type = IntList<>; };

Воспользуйтесь IntList и метафункцией Generate для того, чтобы научиться "раскрывать" кортежи.

От вас потребуется написать функцию apply, которая принимает функтор и кортеж с аргументами для вызова этого функтора и вызывает функтор от этих аргументов.

    // принимает функтор и кортеж с аргументами для вызова этого функтора и вызывает функтор от этих аргументов
    // https://galowicz.de/2016/06/24/integer_sequences_at_compile_time/
    template<int N>
    using seq = typename Generate<N>::type;

    template<typename F, typename TUP, int ...Indexes>
    auto apply(F f, TUP t, IntList<Indexes...>)
    -> decltype(f(std::get<Indexes>(t)...))
    { return f(std::get<Indexes>(t)...); };

    template<typename F, typename ...Ts>
    auto apply(F f, std::tuple<Ts...> t) 
    -> decltype( apply(f, t, seq<sizeof...(Ts)>{}) )
    { return apply(f, t, seq<sizeof...(Ts)>{}); }

Примерно так устроен `std::apply` из С++20

## <a id='toc3_8_'></a>[Вывод списка](#toc0_)

    // вывод списка в поток os
    template<typename TL>
    void printTypeList(std::ostream & os)
    {
        os << typeid(typename TL::Head).name() << ’\n’; // endl флашит поток, если не надо, то \n лучше
        printTypeList<typename TL::Tail>(os);
    };

    // вывод пустого списка
    template<>
    void printTypeList<TypeList<>>(std::ostream & os) {}
    
    int main()
    {
        using TL = TypeList<double, float, int, char>;
        printTypeList<TL>(std::cout);
        return 0;
    }

### <a id='toc3_8_1_'></a>[Задачи](#toc0_)

Напишите метафункцию Zip (аналог std::transform), которая принимает два списка целых чисел одинаковой длины, а так же бинарную метафункцию,  и возвращает список, получившийся в результате поэлементного применения метафункции к соответствующим элементам исходных [списков](https://ru.stackoverflow.com/questions/1181371/%D0%9F%D0%B5%D1%80%D0%B5%D0%B4%D0%B0%D1%87%D0%B0-%D0%B4%D0%B2%D1%83%D1%85-%D0%BD%D0%B0%D0%B1%D0%BE%D1%80%D0%BE%D0%B2-%D0%B0%D1%80%D0%B3%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%BE%D0%B2-%D0%B2-%D0%BC%D0%B5%D1%82%D0%B0%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8E).

    template <typename A, typename B, template <int, int> class Op>
    struct Zip;
    
    template <int ...A, int ...B, template <int, int> class Op>
    struct Zip<IntList<A...>, IntList<B...>, Op>
    {
        using type = IntList<Op<A, B>::value...>;
    };

Как это работает хз

А вот так понятно

    template<class L1, class L2, template<int, int> class F>
    struct Zip
    {
        using type = typename IntCons< F<L1::Head, L2::Head>::value, 
                                    typename Zip<typename L1::Tail,
                                                    typename L2::Tail, F>::type >::type;    
    };

    template<template <int, int> class F>
    struct Zip<IntList<>, IntList<>, F>
    {
        using type = IntList<>;
    };

Определите класс Quantity, которые хранит вещественное число и его размерность в системе СИ (размерность хранится как список IntList длины 7). Для этого класса нужно определить операторы сложения, вычитания, умножения и деления.

Пример:

    template<int m = 0, int kg = 0, int s = 0, int A = 0, int K = 0, int mol = 0, int cd = 0>
    using Dimension = IntList<m, kg, s, A, K, mol, cd>;

    using NumberQ   = Quantity<Dimension<>>;          // число без размерности
    using LengthQ   = Quantity<Dimension<1>>;          // метры
    using MassQ     = Quantity<Dimension<0, 1>>;       // килограммы
    using TimeQ     = Quantity<Dimension<0, 0, 1>>;    // секунды
    using VelocityQ = Quantity<Dimension<1, 0, -1>>;   // метры в секунду
    using AccelQ    = Quantity<Dimension<1, 0, -2>>;   // ускорение, метры в секунду в квадрате
    using ForceQ    = Quantity<Dimension<1, 1, -2>>;   // сила в ньютонах
    ...

    LengthQ   l{30000};      // 30 км
    TimeQ     t{10 * 60};    // 10 минут
    // вычисление скорости
    VelocityQ v = l / t;     // результат типа VelocityQ, 50 м/с

    AccelQ    a{9.8};        // ускорение свободного падения
    MassQ     m{80};         // 80 кг
    // сила притяжения, которая действует на тело массой 80 кг
    ForceQ    f = m * a;     // результат типа ForceQ

Требования к Quantity:
- Конструктор по умолчанию и explicit конструктор от double.
- Метод value(), который возвращает значение типа double.
- Можно складывать только величины одной размерности.
- При умножении (делении) соответствующие размерности поэлементно складываются (вычитаются).
- Нужно реализовать умножение и деление на число типа double.

Указание: для упрощения в данном задании достаточно реализовать только бинарные арифметические операторы. Гарантируется, что все списки, которые будут передаваться в Quantity имеют одинаковую длину.

[Даже не вникал](https://github.com/DGolgovsky/Courses/blob/master/CPP.Part_2/week_5/03.MPL_Basics/Quantity.cpp)

    /* Auxiliary structs*/
    template<int a, int b> struct Plus
    { static int const value = a + b; };
    template<int a, int b> struct Minus
    { static int const value = a - b; };
    /* Auxiliary structs*/

    /* Quantity */
    template<class L>
    class Quantity
    {
    private:
        double val;
    public:
        explicit Quantity(double val = 0)
                :val(val)
        { }
        double value() const noexcept {
            return val;
        }
        Quantity operator+(Quantity const& rhs) const noexcept {
            return Quantity(val + rhs.val);
        }
        Quantity operator-(Quantity const& rhs) const noexcept {
            return Quantity(val - rhs.val);
        }

        template<class B>
        Quantity<typename Zip<L, B, Plus>::type>
            operator*(Quantity<B> const& rhs) const noexcept {
            return Quantity<typename Zip<L, B, Plus>::type>(val * rhs.value());
        }
        Quantity operator*(double val) const noexcept {
            return Quantity<L>(this->val * val);
        }

        template<class B>
        Quantity<typename Zip<L, B, Minus>::type>
            operator/(Quantity<B> const& rhs) const noexcept {
            return Quantity<typename Zip<L, B, Minus>::type>(val / rhs.value());
        }
        Quantity operator/(double val) const noexcept {
            return Quantity<L>(this->val / val);
        }
    };

    template<class L>
    Quantity<L> operator*(double val, Quantity<L> const& rhs) noexcept {
        return Quantity<L>(rhs.value() * val);
    }

    template<class L>
    Quantity<typename Zip<IntList<0,0,0,0,0,0,0>, L, Minus>::type>
        operator/(double val, Quantity<L> const& rhs) noexcept {
        return Quantity<typename Zip<IntList<0, 0, 0, 0, 0, 0, 0>, L, Minus>::type>(val/rhs.value());
    }
    /* Quantity */


# Метапрограммирование: генерация классов и проверка свойств

## Генерация классов

Класс генерируется наследованием. Множественное наследование - это вообще-то очень непростая и неприятная тема. Вкратце - у вас получается несколько экземпляров базового класса, и компилятор просто не понимает, какой именно самый базовый класс имеется в виду. Решений два - в зависимости от того, что вам надо. Либо обращаться к членам этого неоднозначного базового класса с указанием промежуточного класса, либо объявить его виртуальным базовым классом.

Пусть есть 3 класса

    struct A {
        void foo() {std::cout << "struct A\n";}
    };
    struct B {
        void foo() {std::cout << "struct B\n";}
    };
    struct C {
        void foo() {std::cout << "struct C\n";}
    };

Заводим список типов и даем ему имя. Тут список явно задан, но можно получать такой из вычисления на этапе компиляции

    using Bases = TypeList<A, B, C>;

Для наследования от нескольких классов содается шаблонный класс, который наследуется от передаваемого ему списка через `...`, тогда наследование будет вызываться от всех типов из списка

    template<typename TL>
    struct inherit;

    template<typename... Types>
    struct inherit<TypeList<Types...>> : Types... {};
    
И создается уже конкретный потомок этого шаблонного класса

    struct D : inherit<Bases> { };

## Генерация классов (продолжение)

У всех базовых классов в примере был метод с одинаковой сигнатурой, как их вызывать?

Определяется свой метод `foo()` который состоит просто в вызове специализированного шаблонного метода `foo_impl()`, который принимает шаблонный параметр - список типов, который будет соответствовать списку унаследованных классов

    struct D : inherit<Bases>
    {
        void foo() { foo_impl<Bases>(); }
        
        template<typename L> void foo_impl();
    };

*) `foo_impl()` в классе может быть только объявлен, специализация шаблонного метода всегда должна определяться вне класса

Как реализовать такой вызов специализированного метода `foo_impl()`. Используется реализацию по умолчанию + полная специализация для случая пустого списка

Суть в статическом приведение текущего типа this к базовому классу из списка унаследованных. Такой вызов делается для головы списка и рекурсивно повторяется для остальной части списка

    template<typename L>
    inline void D::foo_impl()
    {
        // приводим this к указателю на базу из списка
        static_cast<typename L::Head *>(this)->foo();
        
        // рекурсивный вызов для хвоста списка
        foo_impl<typename L::Tail>();
    }
    
    template<>
    inline void D::foo_impl<TypeList<>>()
    {}

## Как определить наличие метода?

Что если не во всех предках есть нужный метод?

    struct A { void foo() { std::cout << "struct A\n"; } };
    struct B { }; // нет метода foo()
    struct C { void foo() { std::cout << "struct C\n"; } };

    template<typename L>
    inline void D::foo_impl()
    {
        // приводим this к указателю на базу из списка
        static_cast<typename L::Head *>(this)->foo(); // ошибка компиляции(!)

        // рекурсивный вызов для хвоста списка
        foo_impl<typename L::Tail>();
    }

## Как проверить наличие родственных связей?


Определяются структуры для да/нет. Важно, чтобы их размер был гарантированно разным, поэтому уот так уот

    typedef char YES;
    struct NO { YES m[2]; };
    
Метафункция, которая проверяет, является ли класс `B` базовым для некоторого `D`. В ней две перегрузки статической функции `test`, одна принимает указатель на базовый класс и возвращает `YES`, другая - что угодно и возвращает `NO`. Используется тот факт, что приоритет перегрузки с `...` самый последний. Потом сравнивается размер результата этой функции на приведенном к D указателе

    template<class B, class D>
    struct is_base_of
    {
        static YES test(B * );
        static NO test(...);
        
        static bool const value =
            sizeof(YES) == sizeof(test((D *)0));
    };

Если надо исключить срабатывание на одинаковых классов, то уот так уот

    template<class D>
    struct is_base_of<D, D>
    {
        static bool const value = false;
    };

Как вариант  можно воспользоваться `constexpr`

    template<class B, class D>
    struct is_base_of {
        static constexpr bool test(B*) { return true; }
        static constexpr bool test(...) { return false; }

        static const bool value = test((D*)0);
    };

В стандартной библиотеке есть `std::is_base_of` с похожим поведением (с тем отличием, что она покажет ДА, еще и если B и D это один и тот же класс, а пример выше - не покажет). Пример выше также сломается на константности D, не сможет его привести

## SFINAE

Одно из важных правил С++ наз. SFINAE = Substitution Failure Is Not An Error. Ошибка при подстановке шаблонных параметров не является сама по себе ошибкой.

Есть 2 шаблонных функции

    // ожидает, что у типа T определён
    // вложенный тип value_type
    template<class T>
    void foo(typename T::value_type * v);
    
    // работает с любым типом
    template<class T>
    void foo(T t);

Уот такой вызов по правилу SFINAЕ нормально скомпилируется с второй перегрузкой `foo()`

    // при инстанциировании первой перегрузки
    // происходит ошибка (у int нет value_type),
    // но это не приводит к ошибке компиляции
    foo<int>(0);

## Используем SFINAE

Проверялка наличия метода. Включает `wrapper` в котором нам важны только  его шаблонные аргументы: класс Z и указатель на метод Z (`Z::*`) с нужной нам сигнатурой `()`, причем со значением по умолчанию в виде адреса метода в классе Z. Если в шаблон подставить класс без этих условий, то получим ошибку подстановки шаблонных параметров.

Аналогично примеру выше проверяем методом `check` какой вариант этого метода сможет инстанцинироваться

    template<class T>
    struct is_foo_defined
    {
        // обёртка, которая позволит проверить
        // наличие метода foo с заданой сигнатурой
        template<class Z, void (Z::*)() = &Z::foo>
        struct wrapper {};

        template<class C>
        static YES check(wrapper<C> * p);

        template<class C>
        static NO check(...);
        
        static bool const value =
            sizeof(YES) == sizeof(check<T>(0));
    };

## Проверяем наличие метода

С использованной выше проверкой мы можем вызывать методы у базовых классов только если они существуют

Перегрузки шаблонов делаются по типу (мы не можем перегрузить метод, изменить его реализацию на этапе компиляции в зависимости от значения типа, напирмер  bool true/false), поэтому используется такой переходник, преобразующий ответ функции `if_foo_defined` в типы `YES/NO`

    template<bool b>
    struct Bool2Type { using type = YES; };
    
    template<>
    struct Bool2Type<false> { using type = NO; };
    
В результате мы сможем вызвать вспомогательный метод `call_foo` с нужным аргументом, который будет или не будет пытаться вызвать `foo` в классе `Head`

    template<class L>
    void foo_impl()
    {
        using Head = typename L::Head;
    
        constexpr bool has_foo =
            is_foo_defined<Head>::value;
        
        using CALL =
            typename Bool2Type<has_foo>::type;
        
        call_foo<Head>(CALL());
        foo_impl<typename L::Tail>();
    }

После чего в конце рекурсивно вызываемся для остального списка типов

## Проверяем наличие метода (продолжение)

Вспомогательный метод `call_foo`

    struct D : inherit<Bases>
    {
        // ... foo, foo_impl
        template<class Base>
        void call_foo(YES)
        {
            static_cast<Base *>(this)->foo();
        }
        
        template<class Base>
        void call_foo(NO) { }
    };

Нельзя сказать, что такого плана код хоть кто-то использует хоть где-то, скорее это примерная демонстрация как по сути работают аналогичные шаблонные методы стандартной библиотки

## std::enable_if

Есть такой шаблонный класс с частичной специализацией для первого парамета ==true, и тогда он определяет в параметре type второй параметр

    // <type_traits>
    namespace std {
        template<bool B, class T = void>
        struct enable_if {};
    
        template<class T>    
        struct enable_if<true, T> { using type = T; };
    }

Это позволяет использовать SFINAE, если в первый ш.параметр подается false, то происходит ошибка подстановки параметров и данная перегрузка не используется

    template<class T>
    typename std::enable_if<std::is_integral<T>::value, T>::type
    div2(T t) { return t >> 1; }

    template<class T>
    typename std::enable_if<std::is_floating_point<T>::value, T>::type
    div2(T t) { return t / 2.0; }

Получается возможна ситуация, когда у нас как бы 2 функции с одинаковыми аргументами (чего быть не может), но с разными типами возвращаемых значений. По факту у нас для каждого используемого (т.е. к которому применяется функция) типа существует только одна функция `div2`. Вторая отвергается по SFINAE

## std::enable_if (продолжение)

`std::enable_if` позволять организовать условную компиляцию, причем не обязательно ее использвать как выше, в типе возвращаемого значения, 

а засунуть в аргументы функции и выводить их тип

    template<class T>
    T div2(T t, typename std::enable_if<std::is_integral<T>::value, T>::type * = 0)
    { return t >> 1; }

или в шаблонный параметр и выводить его тип

    template<class T, class E = typename std::enable_if<std::is_floating_point<T>::value, T>::type>
    T div2(T t)
    { return t / 2.0; }

Такой подход позволяет использовать `std::enable_if` не только функциями/методами, но и с классами. Определяем ша.класс А с ша.параметрами Т и Е, причем Е используется как проверялка условия для типа Т при частичной специализации класса А. Упороться можно, аж зубы сводит...

    template<class T, class E = void>
    class A;
    
    template<class T>
    class A<T, typename std::enable_if<std::is_integral<T>::value>::type>
    {};

### Задача

Напишите шаблонную функцию get_size, которая принимает значение некоторого типа, у которого есть либо константный метод size() возвращающий size_t, либо поле size типа size_t, и возвращает соответствующее значение.

    std::string s{"Hello"};
    size_t s_size = get_size(s);   // 5, вызывается метод size()

    struct Struct 
    { 
        size_t size = 0;  
    };

    Struct x{10};
    size_t x_size = get_size(x);  // 10, читается поле size

Hint: задача может показаться сложнее, чем на самом деле. Нужно применить SFINAE, но повторять всё, что мы делали для is_foo_defined, не требуется.


[Ожидаемое решение](https://github.com/DGolgovsky/Courses/blob/master/CPP.Part_2/week_5/04.MPL_Gen_Classes/get_size_solutions.txt):

    template<class T, size_t (T::*)() const = &T::size>
    size_t get_size(T const& t)
    {
        return t.size();
    }

    template<class T, size_t (T::*) = &T::size>
    size_t get_size(T const& t)
    {
        return t.size;
    }
